In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn import preprocessing

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

In [2]:
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

In [3]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [4]:
sample.head()

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5


In [5]:
train["kfold"] = -1
n_splits = 5

In [6]:
kf = model_selection.KFold(n_splits=n_splits, shuffle=True, random_state=0)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=train)):
    train.loc[valid_indicies, "kfold"] = fold

In [7]:
useful_features = [c for c in train.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
test = test[useful_features]

In [8]:
for col in object_cols:
    temp_train = []
    temp_test_feat = None
    for fold in range(n_splits):
        xtrain =  train[train.kfold != fold].reset_index(drop=True)
        xvalid = train[train.kfold == fold].reset_index(drop=True)
        feat = xtrain.groupby(col)["target"].agg("mean")
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_train.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = test[col].map(feat)
        else:
            temp_test_feat += test[col].map(feat)
    
    temp_test_feat /= n_splits
    test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    train = pd.concat(temp_train)

In [9]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,tar_enc_cat0,tar_enc_cat1,tar_enc_cat2,tar_enc_cat3,tar_enc_cat4,tar_enc_cat5,tar_enc_cat6,tar_enc_cat7,tar_enc_cat8,tar_enc_cat9
0,22,B,A,A,C,B,D,A,E,A,...,8.247361,8.279035,8.245812,8.237742,8.241678,8.252874,8.24161,8.241407,8.231204,8.255048
1,43,A,B,A,C,B,B,A,E,C,...,8.240164,8.203976,8.245812,8.237742,8.241678,8.230773,8.24161,8.241407,8.282360,8.235549
2,48,A,B,A,C,B,D,A,E,F,...,8.240164,8.203976,8.245812,8.237742,8.241678,8.252874,8.24161,8.241407,8.203212,8.259613
3,50,A,A,A,C,B,B,A,E,E,...,8.240164,8.279035,8.245812,8.237742,8.241678,8.230773,8.24161,8.241407,8.194729,8.259613
4,54,A,B,A,A,B,B,A,E,C,...,8.240164,8.203976,8.245812,8.277061,8.241678,8.230773,8.24161,8.241407,8.282360,8.232367


In [10]:
useful_features = [c for c in train.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
test = test[useful_features]

In [11]:
final_predictions = []
scores = []
for fold in range(5):
    xtrain =  train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state=42, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    scores.append(rmse)
    print(fold, rmse)

print(np.mean(scores), np.std(scores))

0 0.7272601521211494
1 0.7246899183342499
2 0.7291782423860788
3 0.7234396155573898
4 0.7246741068884445
0.7258484070574626 0.0020786621149255727


In [12]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [13]:
output = sample.copy()
output.target = preds
output.to_csv('submission_target_encoding.csv', index=False)